In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast
spark = SparkSession.builder.appName("Jupyter").getOrCreate()
# 

# Disabled automatic broadcast join with spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

# Explicitly broadcast JOINs
medals = spark.read.option("header", "true").csv("/home/iceberg/data/medals.csv")
medals_matches_players = spark.read.option("header", "true").csv("/home/iceberg/data/medals_matches_players.csv")
maps = spark.read.option("header", "true").csv("/home/iceberg/data/maps.csv")
matches = spark.read.option("header", "true").csv("/home/iceberg/data/matches.csv")

print("medals", medals.count())
print("medals_matches_players", medals_matches_players.count())
print("maps", maps.count())
print("matches", matches.count())

medals_matches_players_join_medals = medals_matches_players.join(broadcast(medals), on="medal_id", how="inner")

medals_matches_players_join_medals.explain("FORMATTED")

matches_join_maps = matches.join(broadcast(maps), on="mapid", how="left")

matches_join_maps.explain("FORMATTED")

medals 183
medals_matches_players 755229
maps 183
matches 755229
== Physical Plan ==
AdaptiveSparkPlan (8)
+- Project (7)
   +- BroadcastHashJoin Inner BuildRight (6)
      :- Filter (2)
      :  +- Scan csv  (1)
      +- BroadcastExchange (5)
         +- Filter (4)
            +- Scan csv  (3)


(1) Scan csv 
Output [4]: [match_id#671, player_gamertag#672, medal_id#673, count#674]
Batched: false
Location: InMemoryFileIndex [file:/home/iceberg/data/medals_matches_players.csv]
PushedFilters: [IsNotNull(medal_id)]
ReadSchema: struct<match_id:string,player_gamertag:string,medal_id:string,count:string>

(2) Filter
Input [4]: [match_id#671, player_gamertag#672, medal_id#673, count#674]
Condition : isnotnull(medal_id#673)

(3) Scan csv 
Output [12]: [medal_id#630, sprite_uri#631, sprite_left#632, sprite_top#633, sprite_sheet_width#634, sprite_sheet_height#635, sprite_width#636, sprite_height#637, classification#638, description#639, name#640, difficulty#641]
Batched: false
Location: InMemory